In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
!pip install kaggle==1.5.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73027 sha256=6add3710d675ade0d29ea24f15e48215642570ad5c252a56d5afb0abd2ae76b6
  Stored in directory: /root/.cache/pip/wheels/70/0c/e6/79103212a102e78b8453691b905f48000219574ba7137e7207
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.6.14
    Uninstalling kaggle-1.6.14:
      Successfully uninstalled kaggle-1.6.14


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ozzygomes","key":"8fe22947721bd006058b752c7b3d9955"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d augustop/portuguese-tweets-for-sentiment-analysis

 99% 223M/224M [00:08<00:00, 38.2MB/s]
100% 224M/224M [00:08<00:00, 29.1MB/s]


In [ ]:
!unzip /content/portuguese-tweets-for-sentiment-analysis.zip


Archive:  /content/portuguese-tweets-for-sentiment-analysis.zip
  inflating: NoThemeTweets.csv       
  inflating: TestDatasets/Test.csv   
  inflating: TestDatasets/Test3classes.csv  
  inflating: TestDatasets/TestTema.csv  
  inflating: TrainingDatasets/Train100.csv  
  inflating: TrainingDatasets/Train200.csv  
  inflating: TrainingDatasets/Train300.csv  
  inflating: TrainingDatasets/Train3Classes.csv  
  inflating: TrainingDatasets/Train400.csv  
  inflating: TrainingDatasets/Train50.csv  
  inflating: TrainingDatasets/Train500.csv  
  inflating: TrainingDatasets/TrainTema.csv  
  inflating: TweetsNeutralHash.csv   
  inflating: TweetsNeutralNews.csv   
  inflating: TweetsWithTheme.csv     
  inflating: testdatasets/Test.csv   
  inflating: testdatasets/Test3classes.csv  
  inflating: testdatasets/TestTema.csv  
  inflating: trainingdatasets/Train100.csv  
  inflating: trainingdatasets/Train200.csv  
  inflating: trainingdatasets/Train300.csv  
  inflating: trainingdatasets/Train3

In [ ]:
df = pd.read_csv('/content/NoThemeTweets.csv')

In [ ]:
df.shape

(785814, 5)

In [ ]:
df['sentiment'].value_counts()

sentiment
Negativo    522707
Positivo    263107
Name: count, dtype: int64

como há muitos sentimentos negativos começarei com uma redução dos tweets negativos da base

In [ ]:
#contando o numero de sentimentos positivos e negativos
positive_count = df['sentiment'].value_counts()['Positivo']
negative_count = df['sentiment'].value_counts()['Negativo']

# calculando quantos negativos terie que remover
samples_to_remove = negative_count - positive_count

# randomicamente selecionando os negativos para serem removidos
negative_indices = df[df['sentiment'] == 'Negativo'].index
indices_to_remove = np.random.choice(negative_indices, samples_to_remove, replace=False)

# criando um novo dataaframe sem o excesso de amostras negativoas
df = df.drop(indices_to_remove)

In [ ]:
df['sentiment'].value_counts()

sentiment
Positivo    263107
Negativo    263107
Name: count, dtype: int64

In [ ]:
df.columns
df.shape

(526214, 5)

In [ ]:
df_samples = df.sample(n=100000, random_state=1)

In [ ]:
df_samples.shape

(100000, 5)

#**XGBoost**

In [ ]:


# Remoção de colunas irrelevantes
df = df.drop(columns=['id', 'tweet_date', 'query_used'])

# Tratamento do texto
stop_words_portuguese = ['de', 'a', 'o', 'que', 'e', 'do', 'da', 'em', 'um', 'para',
                         'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                         'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
                         'y', 'z', 'es','ar', 'ag']  # Example list, replace with your own

vectorizer = TfidfVectorizer(stop_words=stop_words_portuguese)
X = vectorizer.fit_transform(df['tweet_text']).toarray()
df['sentiment'] = df['sentiment'].replace({'Negativo': 0, 'Positivo': 1})
# Preparação das labels
y = df['sentiment']

# Divisão dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinamento do modelo XGBoost
model = XGBClassifier(max_depth=8, n_estimators=100)
model.fit(X_train, y_train)

# Avaliação do modelo
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)


In [ ]:


# salvar o modelo com XGBoost
model.save_model('xgboost_model.json')

# Carregar o modelo com XGBoost
# loaded_model_xgb = XGBClassifier()
# loaded_model_xgb.load_model('xgboost_model.json')

In [ ]:
# Função para limpar e preprocessar os tweets
def preprocess_tweet(tweet):
    # Converter para minúsculas
    tweet = tweet.lower()
    # Remover URLs, mencões, hashtags, números e pontuação
    tweet = re.sub(r'http\S+|@\w+|#\w+|\d+', '', tweet)
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    # Remover stopwords e aplicar stemming
    ps = RSLPStemmer()
    tweet = ' '.join([ps.stem(word) for word in tweet.split() if word not in stop_words_portuguese])
    return tweet



# Vetorizar os tweets
vectorizer = CountVectorizer()


#**Novo teste de Modelo**

**Predição Bradesco**

In [ ]:
import pandas as pd
import numpy as np
import re
import string
import nltk
import joblib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
import plotly.express as px

# Downloads necessários do NLTK
nltk.download('stopwords')
nltk.download('rslp')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


True

In [ ]:
stop_words_portuguese = stopwords.words('portuguese')
custom_stop_words = ['es', 'ar', 'ag']  # Stop words adicionais, se necessário
stop_words_portuguese.extend(custom_stop_words)

def preprocess_tweet(tweet):
    # Converter para minúsculas
    tweet = tweet.lower()
    # Remover URLs, mencões, hashtags, números e pontuação
    tweet = re.sub(r'http\S+|@\w+|#\w+|\d+', '', tweet)
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    # Remover stopwords e aplicar stemming
    ps = RSLPStemmer()
    tweet = ' '.join([ps.stem(word) for word in tweet.split() if word not in stop_words_portuguese])
    return tweet

In [ ]:
# Carregar os dados novos
df_bradesco = pd.read_csv('bradesco (1).csv')
df_bradesco.drop_duplicates(inplace=True)

# Preprocessar os tweets novos
df_bradesco['tweet_clean'] = df_bradesco['text'].apply(preprocess_tweet)

# Carregar o modelo e o vetorizador salvos
vectorizer = joblib.load('tfidf_vectorizer.joblib')
X_new = vectorizer.transform(df_bradesco['tweet_clean'])

model = XGBClassifier()
model.load_model('xgboost_model_.json')

# Realizar predições nos dados novos
predictions_bradesco = model.predict(X_new)



In [ ]:
unique_predictions, counts = np.unique(predictions_bradesco, return_counts=True)

fig = px.bar(x=unique_predictions, y=counts,
             labels={'x':'Predictions', 'y':'Count'},
             title='Distribuição das Predições Bradesco')

fig.show()

**Predição Petrobrás**

In [ ]:
# Carregar os dados novos
df_petrobras = pd.read_csv('petrobras.csv')
df_petrobras.drop_duplicates(inplace=True)

# Preprocessar os tweets novos
df_petrobras['tweet_clean'] = df_petrobras['text'].apply(preprocess_tweet)

# Carregar o modelo e o vetorizador salvos
vectorizer = joblib.load('tfidf_vectorizer.joblib')
X_new = vectorizer.transform(df_petrobras['tweet_clean'])

model = XGBClassifier()
model.load_model('xgboost_model_.json')

# Realizar predições nos dados novos
predictions_petrobras = model.predict(X_new)

In [ ]:
unique_predictions, counts = np.unique(predictions_petrobras, return_counts=True)

fig = px.bar(x=unique_predictions, y=counts,
             labels={'x':'Predictions', 'y':'Count'},
             title='Distribuição das Predições Petrobrás')

fig.show()

**Predição BurgerKingBR**

In [ ]:
# Carregar os dados novos
df_BurgerKingBR = pd.read_csv('BurgerKingBR.csv')
df_BurgerKingBR.drop_duplicates(inplace=True)

# Preprocessar os tweets novos
df_BurgerKingBR['tweet_clean'] = df_BurgerKingBR['text'].apply(preprocess_tweet)

# Carregar o modelo e o vetorizador salvos
vectorizer = joblib.load('tfidf_vectorizer.joblib')
X_new = vectorizer.transform(df_BurgerKingBR['tweet_clean'])

model = XGBClassifier()
model.load_model('xgboost_model_.json')

# Realizar predições nos dados novos
predictions_BurgerKingBR = model.predict(X_new)

In [ ]:
unique_predictions, counts = np.unique(predictions_BurgerKingBR, return_counts=True)

fig = px.bar(x=unique_predictions, y=counts,
             labels={'x':'Predictions', 'y':'Count'},
             title='Distribuição das Predições BurgerKingBR')

fig.show()